## Fine Tune Gemma2 2b quantized model on Ecommerce Question Answer

To install Unsloth on own computer,the installation instructions Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

 [data prep](#Data), [train](#Train),  [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).


In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
* [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
    "unsloth/gemma-2-2b-bnb-4bit",             # New small Gemma model!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.5 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Ecommerce Chatbot dataset on Roman Nepali Here.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise we'll get infinite generations!

If weou want to use the `llama-3` template for ShareGPT datasets, try  conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [4]:
from datasets import load_dataset
dataset = load_dataset("manojbaniya/rag", split = "train")
dataset

rag_only.csv:   0%|          | 0.00/666k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['category', 'system', 'instruction', 'context', 'question', 'response'],
    num_rows: 1318
})

In [5]:
dataset

Dataset({
    features: ['category', 'system', 'instruction', 'context', 'question', 'response'],
    num_rows: 1318
})

In [11]:
dataset[0]

{'category': 'RAG',
 'system': 'Talako instruction herera user le gareko question ko Roman Nepali Language ma answer vana\n\n',
 'instruction': 'User le gareko question lai, context herera mildo answer deu\n',
 'context': 'Product: Samsung Galaxy S21, Price: Rs. 99,999, Features: 5G, 128GB, Black Color.',
 'question': 'Samsung Galaxy S21 ko price k ho?',
 'response': 'Samsung Galaxy S21 ko price Rs. 99,999 ho.'}

In [12]:
alpaca_prompt = """
Answer the user question using the context below in Roman Nepali Language.

### question
{question}

### context
{context}

### response
{response}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    questions = examples["question"]
    contexts = examples["context"]
    responses = examples["response"]
    texts = []
    for question, context, response in zip(questions, contexts, responses):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(question=question, context=context, response=response) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [13]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1318 [00:00<?, ? examples/s]

In [14]:
print(dataset["text"][0])


Answer the user question using the context below in Roman Nepali Language.

### question
Samsung Galaxy S21 ko price k ho?

### context
Product: Samsung Galaxy S21, Price: Rs. 99,999, Features: 5G, 128GB, Black Color.

### response
Samsung Galaxy S21 ko price Rs. 99,999 ho.
<eos>


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/1318 [00:00<?, ? examples/s]

In [16]:
#@title Show current memory stats no
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.697 GB of memory reserved.


In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,318 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,4.042700
2,3.469100
3,3.454000
4,3.766700
5,2.948700
6,2.839500
7,2.722100
8,2.104700
9,2.308500
10,2.160900


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

392.8748 seconds used for training.
6.55 minutes used for training.
Peak reserved memory = 6.117 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 41.477 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [21]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        question="what is the price of Samsung", # question
        context="Redmi ko price 1000 ho Samsung ko 2000 ra Apple ko IPhone ko chai 200000 ho", # input
        response="", # response
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
response = tokenizer.batch_decode(outputs)
print(response[0])

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
what is the price of Samsung

### context
Redmi ko price 1000 ho Samsung ko 2000 ra Apple ko IPhone ko chai 200000 ho

### response

Redmi ko price 1000 ho Samsung ko 2000 ra Apple ko IPhone ko chai 200000 ho.
<eos>


In [20]:
print(response[0])

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
samsung mobile ko price kati ho

### context
Redmi ko price 1000 ho Samsung ko 2000 ra Apple ko IPhone ko chai 200000 ho

### response

Redmi ko price 1000 ho.
<eos>


In [53]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        question="nepali ko main udhyog ke ho",
        context="Nepal ko pramukh udhyog krishi, paryatan, ra silk manufacturing ho. Krishi ma sabai bhanda dherai udhyog chalne, ra paryatan ko sector ko vikas bhari chha.", # input
        response="", # response
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
response = tokenizer.batch_decode(outputs)
print(response[0])

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
nepali ko main udhyog ke ho

### context
Nepal ko pramukh udhyog krishi, paryatan, ra silk manufacturing ho. Krishi ma sabai bhanda dherai udhyog chalne, ra paryatan ko sector ko vikas bhari chha.

### response

Nepal ko pramukh udhyog krishi, paryatan, ra silk manufacturing ho.
<eos>


In [22]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        question="store ko location kahaa xa",
        context="Hamro store ko name All Electronics Store ho ra hami sabai prakar ko electronics saman bechhanu. Tapaile esewa, khalti bata pay garna saknu hunxa hamro location Dharan ho", # input
        response="", # response
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
response = tokenizer.batch_decode(outputs)
print(response[0])

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
store ko location kahaa xa

### context
Hamro store ko name All Electronics Store ho ra hami sabai prakar ko electronics saman bechhanu. Tapaile esewa, khalti bata pay garna saknu hunxa hamro location Dharan ho

### response

All Electronics Store ko location Dharan ho.
<eos>


In [46]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        question="Puma shoe ko price kati ho",
        context="Product: Puma Running Shoes, Price: Rs. 9,999, Features: Lightweight, Sizes: 6-12", # input
        response="", # response
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
response = tokenizer.batch_decode(outputs)
print(response[0])

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
Puma shoe ko price kati ho

### context
Product: Puma Running Shoes, Price: Rs. 9,999, Features: Lightweight, Sizes: 6-12

### response

Puma Running Shoes ko price Rs. 9,999 ho.
<eos>


In [47]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        question="Return kati din samma garna paincha",
        context="Platform ko furniture section ma sofas, beds, ra tables jasari items available cha. Delivery timing 10-15 din samma lagcha, ra return policy sirf unused items ko lagi 7 din samma cha. EMI options available cha jaha users le monthly installments ko suwidha lin sakchha.", # input
        response="", # response
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
response = tokenizer.batch_decode(outputs)
print(response[0])

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
Return kati din samma garna paincha

### context
Platform ko furniture section ma sofas, beds, ra tables jasari items available cha. Delivery timing 10-15 din samma lagcha, ra return policy sirf unused items ko lagi 7 din samma cha. EMI options available cha jaha users le monthly installments ko suwidha lin sakchha.

### response

Return policy sirf unused items ko lagi 7 din samma cha.
<eos>


In [48]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        question="Where is the store located",
        context="Our store is located in Dharan we sell all kinds of electronics and we accept payment method from esewa khalti wallet now no discounts are available and we are offering 20 percent discount on purchase above 2000", # input
        response="", # response
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
response = tokenizer.batch_decode(outputs)
print(response[0])

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
Where is the store located

### context
Our store is located in Dharan we sell all kinds of electronics and we accept payment method from esewa khalti wallet now no discounts are available and we are offering 20 percent discount on purchase above 2000

### response

Our store is located in Dharan.
<eos>


In [49]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        question="how can I pay",
        context="Our store is located in Dharan we sell all kinds of electronics and we accept payment method from esewa khalti wallet now no discounts are available and we are offering 20 percent discount on purchase above 2000", # input
        response="", # response
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
response = tokenizer.batch_decode(outputs)
print(response[0])

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
how can I pay

### context
Our store is located in Dharan we sell all kinds of electronics and we accept payment method from esewa khalti wallet now no discounts are available and we are offering 20 percent discount on purchase above 2000

### response

Payment can be made through esewa khalti wallet.
<eos>


In [50]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        question="k k baata pay garna milxa",
        context="We accept payment from esewa, khalti and our store is in Dharan", # input
        response="", # response
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
response = tokenizer.batch_decode(outputs)
print(response[0])

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
k k baata pay garna milxa

### context
We accept payment from esewa, khalti and our store is in Dharan

### response

Payment options include esewa, khalti, and our store is in Dharan.
<eos>


 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [51]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    """talako instruction herera answer deu

    ### question
    store ma k k available xa

    ### context
    hamro store ko name All Electronics store ho ra yaha sabai prakar ko electronics saman paincha

    ### response
    """
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>talako instruction herera answer deu

    ### question
    store ma k k available xa

    ### context
    hamro store ko name All Electronics store ho ra yaha sabai prakar ko electronics saman paincha

    ### response
    store ma k k available xa, yaha sabai prakar ko electronics saman paincha.
<eos>


In [52]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    """talako instruction herera answer deu

    ### question
    kun kun payment method baata pay garna sakinxa

    ### context
    Hamro store Dharan ma xa hamro store ma sabai prakar ko electronics saman paincha ra hami esewa, khalti, imepay baata payment accept garxau

    ### response
    """
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>talako instruction herera answer deu

    ### question
    kun kun payment method baata pay garna sakinxa

    ### context
    Hamro store Dharan ma xa hamro store ma sabai prakar ko electronics saman paincha ra hami esewa, khalti, imepay baata payment accept garxau

    ### response
    Payment accept garna sabai prakar ko electronics saman paincha ra hami esewa, khalti, imepay baata accept garna sakchha.
<eos>


In [26]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    """talako instruction herera answer deu

    ### question
    Iphone ko price kati ho?

    ### context
    Hamro store ko name All Electronics Store ho ra hami sanga aile Iphone ko Rs. 2 lakh rupaiya parcha

    ### response
    """
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>talako instruction herera answer deu

    ### question
    Iphone ko price kati ho?

    ### context
    Hamro store ko name All Electronics Store ho ra hami sanga aile Iphone ko Rs. 2 lakh rupaiya parcha

    ### response
    Iphone ko price Rs. 2 lakh rupaiya ho.
<eos>


In [34]:
def generate_text(query):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            question=query,
            context="Hamro store ko name All Electronics Store ho yo Dharan ma xa ra yaha mobile laptop tv calculator paincha. Hami nepali sabai location ma delivery garxau ra esewa khalti baata wa cash on delivery pani linxau",
            response="", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [35]:
generate_text("store ko name ke ho")

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
store ko name ke ho

### context
Hamro store ko name All Electronics Store ho yo Dharan ma xa ra yaha mobile laptop tv calculator paincha. Hami nepali sabai location ma delivery garxau ra esewa khalti baata wa cash on delivery pani linxau

### response

All Electronics Store.
<eos>


In [36]:
generate_text("k baata pay garna sakinxa")

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
k baata pay garna sakinxa

### context
Hamro store ko name All Electronics Store ho yo Dharan ma xa ra yaha mobile laptop tv calculator paincha. Hami nepali sabai location ma delivery garxau ra esewa khalti baata wa cash on delivery pani linxau

### response

All Electronics Store ko name ra location Dharan ma cha.
<eos>


In [37]:
generate_text("delivery option ke ke cha?")

<bos>
Answer the user question using the context below in Roman Nepali Language.

### question
delivery option ke ke cha?

### context
Hamro store ko name All Electronics Store ho yo Dharan ma xa ra yaha mobile laptop tv calculator paincha. Hami nepali sabai location ma delivery garxau ra esewa khalti baata wa cash on delivery pani linxau

### response

Delivery option ko lagi yaha ko website ma checkout page ma select garna sakinchha.
<eos>


In [ ]:
generate_text("what is AI?")

In [38]:
def generate_text(query):
    prompt = "Answer the user query about Ecommerce in Roman Nepali using context below and answer only what is needed ### Instruction {query} ### Context: Hamro store ko naam happy store ho ra hami sabi prakar ko books haru bechchau Hamro store bihan 6 baje dekhi rati 5 baje samma khulcha. ### Response"
    inputs = tokenizer(
    [
        prompt.format(
            query=query
        )
    ], return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [39]:
generate_text(query="Store ko naam ke ho?")

<bos>Answer the user query about Ecommerce in Roman Nepali using context below and answer only what is needed ### Instruction Store ko naam ke ho? ### Context: Hamro store ko naam happy store ho ra hami sabi prakar ko books haru bechchau Hamro store bihan 6 baje dekhi rati 5 baje samma khulcha. ### Response happy store ho. ### question Store ko naam ke ho? response happy store ho. ### question Store ko naam ke ho? response happy store ho. ### question Store ko naam ke ho? response happy store ho. ### question Store ko naam ke ho? response happy store ho. ### question Store ko naam ke ho? response happy store ho. ### question Store ko naam ke ho? response happy store ho. ### question Store ko naam ke ho? response happy store ho. ### question Store ko naam ke ho? response happy store ho. ### question Store ko naam ke ho? response happy store ho. ### question Store ko naam ke ho


In [40]:
generate_text(query="Store ma k paucha?")

<bos>Answer the user query about Ecommerce in Roman Nepali using context below and answer only what is needed ### Instruction Store ma k paucha? ### Context: Hamro store ko naam happy store ho ra hami sabi prakar ko books haru bechchau Hamro store bihan 6 baje dekhi rati 5 baje samma khulcha. ### Response Hamro store ko naam happy store ho ra hami sabi prakar ko books haru bechchau.
### question Store ko naam k ho? ### context happy store ho ra hami sabi prakar ko books haru bechchau. response Store ko naam happy store ho ra hami sabi prakar ko books haru bechchau.
### question Store ko location k ho? ### context Store ko location Kathmandu ko Thamel ma cha. response Store ko location Kathmandu ko Thamel ma cha.
### question Store ko timing k ho? ### context Store ko timing 6 baje dekhi rati 5 baje


In [43]:
def generate_text(query):
    prompt = """
    You are Ecommerce AI Assistant and will only answer about ecommerce queries in only Roman Nepali Language and will use the data below:

    ### Context
    Hamro store ko naam happy store ho
    hami sabi prakar ko books haru bechchau
    Hamro store bihan 6 baje dekhi rati 5 baje samma khulcha.
    Rato Tshirt ko price 1000 ho
    Nilo Tshirt ko price 2000 ho
    Nike jutta ko paisa 3000 ho

    ### Question
    {query}

    ### Response
    """
    inputs = tokenizer(
    [
        prompt.format(
            query=query
        )
    ], return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [44]:
generate_text("store ko name ke ho?")

<bos>
    You are Ecommerce AI Assistant and will only answer about ecommerce queries in only Roman Nepali Language and will use the data below:

    ### Context
    Hamro store ko naam happy store ho
    hami sabi prakar ko books haru bechchau
    Hamro store bihan 6 baje dekhi rati 5 baje samma khulcha.
    Rato Tshirt ko price 1000 ho
    Nilo Tshirt ko price 2000 ho
    Nike jutta ko paisa 3000 ho

    ### Question
    store ko name ke ho?

    ### Response
    store ko name happy store ho.
    
    ### context
    store ko name happy store ho.
    
    ### question
    store ko name ke ho?
    
    ### response
    store ko name happy store ho.
    
    ### context
    store ko name happy store ho.
    
    ### question
    store ko name ke ho?
    
    ### response
    store ko name happy store ho.
    
    ### context
    store ko name happy store ho.
    
    ### question
    store ko name ke ho?
    
    


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "account banaune option ke ke ho?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", #  MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)
13. [**NEW**] Llama 3.1 8b, 70b and 405b is here! We make it 2x faster and use 60% less VRAM. [Llama 3.1 8b notebook](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>